In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# OS / data
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# CRUD module
from CRUD_Python_Module import AnimalShelter

# --------------------------
# Data Manipulation / Model
# --------------------------
username = "aacuser"
password = "MyStrongPassword123"

db = AnimalShelter(username, password)

# read all docs once for initial table render
df = pd.DataFrame.from_records(db.read({}))
if not df.empty and "_id" in df.columns:
    df.drop(columns=["_id"], inplace=True)

# --------------------------
# Dashboard Layout / View
# --------------------------
app = JupyterDash(__name__)

# Optional logo
image_filename = "Grazioso Salvare Logo.png"
encoded_image = None
if os.path.exists(image_filename):
    encoded_image = base64.b64encode(open(image_filename, "rb").read()).decode()

app.layout = html.Div([
    html.Center(html.B(html.H1("CS-340 Dashboard"))),

    html.Div(
        [
            html.Img(
                src=f"data:image/png;base64,{encoded_image}",
                style={"height": "60px"},
            ) if encoded_image else html.Div(),
            html.Div(
                "Unique Identifier: Jimmy - Project 2",
                style={"fontStyle": "italic"},
            ),
        ],
        style={"textAlign": "center"},
    ),

    html.Hr(),

    # ---- controls (interactive filtering options) ----
    html.Div(
        children=[
            html.Label("Rescue Type"),
            dcc.RadioItems(
                id="filter-type",
                options=[
                    {"label": "Water Rescue", "value": "Water Rescue"},
                    {"label": "Mountain or Wilderness Rescue", "value": "Mountain or Wilderness Rescue"},
                    {"label": "Disaster or Individual Tracking", "value": "Disaster or Individual Tracking"},
                    {"label": "Reset (All)", "value": "Reset"},
                ],
                value="Reset",
                inline=True,
            ),
        ],
        style={"background": "#f7f7f7", "padding": "12px", "borderRadius": "8px"},
    ),

    html.Hr(),

    # ---- single interactive data table (only one with this id) ----
    dash_table.DataTable(
        id="datatable-id",
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict("records"),
        column_selectable="multi",
        selected_columns=[],
        row_selectable="single",
        selected_rows=[0],
        page_size=12,
        sort_action="native",
        filter_action="native",
        style_table={"overflowX": "auto"},
        style_header={
            "backgroundColor": "#f1f1f1",
            "fontWeight": "bold",
            "textAlign": "center",
        },
        style_cell={
            "textAlign": "left",
            "padding": "8px",
            "minWidth": "120px",
            "whiteSpace": "normal",
        },
    ),

    html.Br(),
    html.Hr(),

    # charts row
    html.Div(
        className="row",
        style={"display": "flex"},
        children=[
            html.Div(id="graph-id", className="col s12 m6"),
            html.Div(id="map-id", className="col s12 m6"),
        ],
    ),
])

# ---------------------------------------------
# Interaction Between Components / Controller
# ---------------------------------------------

# 1) Filter table data based on rescue type
@app.callback(
    Output("datatable-id", "data"),
    [Input("filter-type", "value")]
)
def update_dashboard(filter_type):
    RESCUE_BREEDS = {
        "Water Rescue": [
            "Labrador Retriever", "Labrador Retriever Mix",
            "Chesapeake Bay Retriever", "Newfoundland",
        ],
        "Mountain or Wilderness Rescue": [
            "German Shepherd", "German Shepherd Dog Mix",
            "Siberian Husky", "Alaskan Malamute",
        ],
        "Disaster or Individual Tracking": [
            "Doberman Pinscher", "German Shepherd", "Rottweiler",
            "Bloodhound", "Bluetick Coonhound",
            "Black and Tan Coonhound", "Plott Hound",
        ],
        
    }

    MAX_AGE_WEEKS = 104 # <= 2 years old

    if not filter_type or filter_type.startswith("Reset"):
        query = {}
    else:
        breeds = RESCUE_BREEDS.get(filter_type, [])
        query = {
            "animal_type": {"$regex": "Dog", "$options": "i"},
            "breed": {"$in": breeds},
            "age_upon_outcome_in_weeks": {"$lte": MAX_AGE_WEEKS},
        }

    try:
        data = db.read(query)
        # drop _id if present
        if data and "_id" in data[0]:
            for d in data:
                d.pop("_id", None)
        return data
    except Exception as e:
        print("READ error:", e)
        return []

# 2) Chart from current (visible) table data
@app.callback(
    Output("graph-id", "children"),
    [Input("datatable-id", "derived_virtual_data")]
)
def update_graphs(viewData):
    if not viewData:
        return [dcc.Graph(figure=px.bar(title="Breed Counts (no data)"))]

    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty or "breed" not in dff.columns:
        return [dcc.Graph(figure=px.bar(title="Breed Counts (no data)"))]

    counts = dff["breed"].value_counts().reset_index()
    counts.columns = ["breed", "count"]
    fig = px.bar(counts.head(15), x="breed", y="count", title="Breed Counts")
    fig.update_layout(xaxis_tickangle=-40, margin=dict(l=20, r=20, t=40, b=80))
    return [dcc.Graph(figure=fig)]

# 3) Shade selected columns (optional nicety)
@app.callback(
    Output("datatable-id", "style_data_conditional"),
    [Input("datatable-id", "selected_columns")]
)
def update_styles(selected_columns):
    return [{
        "if": {"column_id": c},
        "background_color": "#D2F3FF",
    } for c in (selected_columns or [])]

# 4) Geo map for selected row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    # If there’s no data or no selection, don’t render anything
    if viewData is None or index is None or len(index) == 0:
        return []

    # Convert the data to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)

    # Get the selected row index
    row = index[0]

    # Make sure we don’t go out of range
    if row >= len(dff):
        return []

    # Create the map
    return [
        dl.Map(
            style={'width': '100%', 'height': '500px'},
            center=[30.75, -97.48], # Default center (Austin, TX)
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                    children=[
                        dl.Tooltip(dff.iloc[row, 4]),
                        dl.Popup([
                            html.H4("Animal Name"),
                            html.P(dff.iloc[row, 9])
                        ])
                    ]
                )
            ]
        )
    ]


# Run app
app.run_server(port=8061)

Dash app running on https://crystallogo-cupidrichard-3000.codio.io/proxy/8061/
